# 1. Fundamentos de *Machine Learning*

En este notebook se revisarán los conceptos de:

1. Notación
2. Vecinos más próximos
3. Repaso de Pandas
4. Evaluación del modelo: entrenamiento y test
5. Selección del modelo: validación cruzada
6. Conceptos fundamentales de ML
  1. Compromiso sesgo-varianza
  2. Curvas de aprendizaje

Primero cargamos librerías y funciones necesarias, incluyendo las del módulo `utils`:

In [ ]:
from utils import plot_decision_boundary, poly_linear_regression, CM_BRIGHT

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline 

## 1. Notación

Vamos a importar la librería principal de este módulo, scikit-learn. Habitualmente se importa como `sklearn`.

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
df = pd.read_csv("./data/automobile.csv")

In [ ]:
df.shape

In [ ]:
# show the first 5 rows using dataframe.head() method
print("Las primeras 5 filas del dataframe") 
df.head(5)

In [ ]:
index = df.index
columns = df.columns
values = df.values

print(type(index))
print(type(columns))
print(type(values))

In [ ]:
df["make"].head(5)

In [ ]:
df[["fuel-system", "make"]].tail(5) # we dont have to follow the original column order when subsetting

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
# describe all the columns in "df" 
df.describe(include = "all")

In [ ]:
# describe returns a df, so we can still perfom operations to all the columns in "df" 
df.describe(include = "all").fillna("-")
# pandas user guide: https://pandas.pydata.org/docs/user_guide/index.html

## Datasets

Podemos echar un vistazo a los [datasets](http://scikit-learn.org/stable/datasets) de sklearn.

In [ ]:
from sklearn import datasets

diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

<div class = "alert alert-success">
EJERCICIO 1.1: Sobre el conjunto de datos anterior, el dataset de diabetes, vamos a calcular los siguientes valores:
</div>

* $N$: número de muestras
* $d$: dimensionalidad del espacio de entrada
* $\mathbf{x}^{(10)}$: muestra $i=10$
* $\mathbf{x}_1$: característica/variable/*feature* $1$ 

In [ ]:
# Para resolver el problema, vamos a seguir una serie de pasos.
# Lo primero es saber a qué nos enfrentamos: qué son x e y?

# ... código para analizar qué tipo de datos son X e y
print(type(X))
print(type(y))

# .... código para saber el tamaño (o la forma) de X e y
print(X.shape)
print(y.shape)

# Sabiendo la forma de X deberíamos ser capaces de determinar el número de muestras y la dimensionalidad

n = ...
d = ...

print(f'El numero de muestras es {n} y la dimensionalidad es {d}')

In [ ]:
# En cuanto a la muestra número 10, debemos recordar que Python es zero-indexed

# ... código para extraer el décimo elemento en la primera dimensión de X (es decir, las filas)

...

In [ ]:
# Para la primera característica, también debemos recordar que Python es zero-indexed

# ... código para extraer el primer elemento en la segunda dimensión de X (es decir, las columnas)

...

<div class = "alert alert-success">
EJERCICIO 1.2: ¿Es un problema de clasificación o de regresión? ¿Por qué?
</div>

In [ ]:
# ¿Cómo podríamos determinar si el problema es de regresión o clasificación? ¿Qué es lo que diferencia a uno de otro?

# ... código

...

## Regresion Lineal

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
print(iris.DESCR)

In [ ]:
# https://en.wikipedia.org/wiki/Iris_flower_data_set

In [ ]:
df = iris.data
df

In [ ]:
df["type"] = iris.target
df

In [ ]:
df["type_name"] = df["type"].map({0:"Setosa",1:"Versicolour",2:"Virginica"})
df.columns = ["sepal_l", "sepal_w", "petal_l", "petal_w", "type", "type_name"]
df

In [ ]:
df.plot.scatter(x="sepal_l", y="petal_l",c="type", colormap='viridis')

In [ ]:
import seaborn as sns

df_plot = df.drop(columns=["type"])
sns.pairplot(df_plot, hue="type_name")

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
X=df[["sepal_l", "sepal_w", "petal_l", "petal_w"]]
y=df["type"]
lr = LinearRegression().fit(X,y)

In [ ]:
# Crear el pairplot
g = sns.pairplot(df_plot, hue="type_name", height=2.5)

# Función para agregar línea de tendencia
def add_trendline(x, y, color, ax):
    m, b = np.polyfit(x, y, 1)
    ax.plot(x, m*x + b, color=color)

# Iterar sobre los subplots del pairplot
variables = ["sepal_l", "sepal_w", "petal_l", "petal_w"]
for i, var1 in enumerate(variables):
    for j, var2 in enumerate(variables):
        if i != j:  # Evitar la diagonal
            ax = g.axes[i, j]
            for t, color in zip(df["type_name"].unique(), sns.color_palette()):
                data = df[df["type_name"] == t]
                add_trendline(data[var2], data[var1], color, ax=ax)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# La regresion polinomica es solamente un caso especial de regresion lineal
# Asi que solo tenemos que transformar los datos para adaptarlos a
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(X)
poly_features

In [ ]:
lr2 = LinearRegression().fit(poly_features, y)
lr2

In [ ]:
from sklearn.pipeline import make_pipeline

# Crear el pairplot
g = sns.pairplot(df_plot, hue="type_name", height=2.5)

# Función para agregar línea de tendencia
def add_poly_regression(x, y, color, ax, degree):
    x_array = x.values.reshape(-1, 1)
    
    # Crear y ajustar el modelo polinomial
    poly_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    poly_model.fit(x_array, y)
    
    # Generar puntos para la línea suave
    x_plot = np.linspace(x.min(), x.max(), 100).reshape(-1, 1)
    y_plot = poly_model.predict(x_plot)
    
    # Dibujar la línea de regresión polinomial
    ax.plot(x_plot, y_plot, color=color, alpha=0.8)

# Iterar sobre los subplots del pairplot
variables = ["sepal_l", "sepal_w", "petal_l", "petal_w"]
for i, var1 in enumerate(variables):
    for j, var2 in enumerate(variables):
        if i != j:  # Evitar la diagonal
            ax = g.axes[i, j]
            for t, color in zip(df["type"].unique(), sns.color_palette()):
                data = df[df["type"] == t]
                add_poly_regression(data[var2], data[var1], color, ax, degree=2)

plt.tight_layout()
plt.show()

<div class = "alert alert-success">
EJERCICIO: Entrena una regresion de grado 4 y grafica los resultados. Dependiendo de los datos que tengamos, ¿que debemos esperar del modelo que elijamos?
</div>

In [ ]:
# ... code


## Calcular Error

En sklean es muy sencillo calcular el error de una regresion lineal. Solamente hay que llamar al metodo `.score()`

In [ ]:
X=df[["sepal_l", "sepal_w", "petal_l", "petal_w"]]
lr.score(X,y)

<div class = "alert alert-success">
EJERCICIO: Calcula el error para todas las regresiones polinomicas hasta grado 10, ¿Cual es la regresion con menos error? ¿Que termino parece ser el que mas afecta al error?

Por que pasa esto?
</div>

In [ ]:
errors = []
for degree in range(1,11):
    # ... code
    ...
    
errors
#_ = pd.DataFrame(errors).plot(title="Error R2")

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

## Métricas

In [ ]:
from sklearn.metrics import root_mean_squared_error

def accuracy(y_true_m, y_pred_m):
    assert len(y_true_m)==len(y_pred_m), "Error: Las longitudes no son iguales."
    correct = np.sum(y_true_m.values == y_pred_m)
    return correct/len(y_true_m)


lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)

root_mean_squared_error(y_true=y_test, y_pred=y_pred)

<div class = "alert alert-success">
EJERCICIO: Similar al ejercicio anterior, pero realiza el train/test split y utiliza la metrica RMSE.
Que observas esta vez?
</div>

Nota: Esta vez lo haremos bien. Utiliza `sepal_w, petal_w, petal_l` como `X` y `sepal_l` como `y`. ¿Puedes explicar en que clase de problema nos enfrentamos? ¿Si esto fuera el mundo real, y sabiendo que representa el Iris dataset, que estariamos intentando hacer?

In [ ]:
# ... code

errors = []
for degree in range(1,11):
    # ... code
    ...

errors
#_ = pd.DataFrame(errors).plot(title="Error RMSE")

## 2. Vecinos más próximos

En este notebook vamos a trabajar con el algoritmo de KNN en distintos problemas de **clasificación**.

### 2.1. Medida de las prestaciones de un clasificador

Por clasificador entendemos un algoritmo que, a partir de un conjunto de muestras/observaciones de entrenamiento, es capaz de identificar a qué clase (categoría) pertenece una nueva observación.

Una métrica de calidad que podemos usar para medir las prestaciones de un clasificador es el **error de clasificación**

$$\textrm{Error} = \frac{\textrm{núm de muestras mal clasificadas}}{\textrm{núm de muestras total del problema}}$$

* Ejemplo: problema de clasificación con dos clases $y\in{0,1}$
    * Etiquetas reales (*y_true*) = $[1,0,0,1,0]$
    * Etiquetas predichas (*y_pred*) = $[0,0,1,1,0]$
    
    * En este caso: $$\textrm{Error} = \frac{\textrm{núm de muestras mal clasificadas} = 2}{\textrm{núm de muestras total del problema} = 5} = \frac{2}{5} = 0.4$$

Así, el error de clasificación será un número entre 0 y 1, tal que:

* $\textrm{Error} = 0$ es el mejor valor posible (no me equivoco nada)
* $\textrm{Error} = 1$ es el peor valor posible (me equivoco en todas las muestras). Nota: si me equivoco en la clasificación de todas las muestras, entonces puedo interpretar que el clasificador es bueno, pero que tengo que hacer justo lo contrario de lo que me dice. El peor valor de error sería por tanto $0.5$, en el que la incertidumbre es mayor. 

Normalmente no se utiliza el error, sino su complementario, la exactitud o [**accuracy**](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) (Acc):

$$\textrm{Acc} = 1 - \textrm{Error}$$

y entonces:

* $\textrm{Acc} = 1$ es el mejor valor posible (no me equivoco nada)
* $\textrm{Acc} = 0$ es el peor valor posible (me equivoco en todas las muestras)

### 2.2 Ejemplos

Para analizar el comportamiento del algoritmo de K-NN, utilizaremos tres ejemplos sencillos, como mostraremos a continuación

In [ ]:
# ejemplo1
ejemplo1 = pd.read_csv("./data/ex2data1.txt", sep=",", header=None, names=['x1', 'x2', 'label'])
ejemplo1.head()

#### 2.2.1 Pequeño desvío: repaso de Pandas

Vamos a hacer un pequeño repaso de las funciones más habituales. No toméis esto como un estudio exhaustivo, ni mucho menos; pero grosso modo os servirá para este módulo.

Vamos a ver los siguientes métodos:

- `.describe()`, que proporciona un pequeño análisis estadístico. El parámetro `include=all` permite añadir variables categóricas
- `.shape`
- `.head()`
- `.tail()`
- `.dtypes`
- Análisis de valores nulos con `.isnull()` e `.isnull().any()`
- Eliminación de columnas con `.drop(c1, axis=1)`
- Cómo acceder a los índices internos, con `.index` y `.index.values`
- Cómo acceder a un elemento determinado en base a su índice, con `.iloc[[i1, i2, i3, ...]]`
- Cómo construir un nuevo dataframe filtrando el anterior, con `df_filtered = df[condición]`
- Cómo construir un nuevo dataframe filtrando el anterior bajo condición múltiple, con `df_filtered = df[(condición 1) & (condición 2)]`


In [ ]:
df = ejemplo1
df.describe(include='all')

In [ ]:
df.shape

In [ ]:
df.head(7)

In [ ]:
df.tail(7)

In [ ]:
df.dtypes

In [ ]:
df.isnull().any()

In [ ]:
df_to_drop = df.drop('x1', axis=1)
df_to_drop.head()

In [ ]:
df.index.values

In [ ]:
df.iloc[[2, 45, 23, 98]]

In [ ]:
df_filtered = df[(df['x1'] > 45) & (df['x2'] < 60)]
df_filtered.head()

Una vez estudiado el dataframe, podemos representar:

In [ ]:
plt.scatter(ejemplo1['x1'], ejemplo1['x2'], c=ejemplo1['label'], cmap=CM_BRIGHT)
plt.xlabel("$x_1$", fontsize=16)
plt.ylabel("$x_2$", fontsize=16)
plt.show()

Se dice que este problema es **linealmente separable**, porque podemos trazar una recta para separar las dos clases (representadas en distintos colores, rojo y azul).
* En el plano bidimensional: recta
* En un espacio d-dimensional: hiperplano

Nota: No es linealmente separable puesto que la separación no es perfecta. Pero es _casi_ linealmente separable, aceptamos.

In [ ]:
# ejemplo2
ejemplo2 = pd.read_csv("./data/ex2data2.txt", sep=",", header=None, names=['x1', 'x2', 'label'])

plt.scatter(ejemplo2['x1'], ejemplo2['x2'], c=ejemplo2['label'], cmap=CM_BRIGHT)
plt.xlabel("$x_1$", fontsize=16)
plt.ylabel("$x_2$", fontsize=16)
plt.show()

Se dice que este problema es **no linealmente separable**, porque no podemos trazar una recta para separar las dos clase (representadas en distintos colores, rojo y azul).

In [ ]:
# ejemplo 3: Problema XOR 
np.random.seed(0)

# -- parameters
N     = 800
mu    = 1.5      # Cambia este valor
sigma = 1      # Cambia este valor

# variables auxiliares
unos = np.ones(int(N/4))
random4 = sigma*np.random.randn(int(N/4),1)
random2 = sigma*np.random.randn(int(N/2),1)

# -- features
y3 = np.concatenate([-1*unos, unos, unos, -1*unos]) 
X1 = np.concatenate([-mu + random4, mu + random4, -mu + random4, mu + random4])
X2 = np.concatenate([+mu + random2, -mu + random2])
X3 = np.hstack((X1,X2))

plt.scatter(X3[:,0], X3[:,1], c=y3, cmap=CM_BRIGHT)
plt.xlabel("$x_1$", fontsize=16)
plt.ylabel("$x_2$", fontsize=16)
plt.show()

Al igual que en el caso anterior, este ejemplo tampoco es linealmente separable, y se conoce como problema XOR. La ventaja del problema XOR es que conocemos cuál es la frontera de separación óptima a priori:

- Clase 1, color azul: $x_1,x_2 > 0$, y $ x_1,x_2 < 0$ (cuadrantes 1 y 3)
- Clase 2, color rojo: $x_1 < 0,  x_2 > 0$, y $x_1 > 0,  x_2 < 0$ (cuadrantes 2 y 4)

## 2.3 Entrenar el modelo 

Vamos a entrenar un modelo K-NN (<a href="http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html">documentación</a> aquí) para los distintos ejemplos:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Ejemplo 1
# preparamos los datos
data1 = ejemplo1.values
print(f'El tipo de datos es {type(data1)}')
X1 = data1[:, 0:2]
y1 = data1[:, -1]

# creamos el modelo y ajustamos
knnModel = KNeighborsClassifier(n_neighbors=10).fit(X1, y1)

plot_decision_boundary(X1, y1, knnModel)

**Número de vecinos**

Podemos modificar el número de vecinos $k$ del algoritmo k-nn implementado en scikit-learn mediante el parámetro *n_neighbors*. Por defecto, [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) toma *n_neighbors* $=5$

<div class = "alert alert-success">
EJERCICIO 1.3: Varía el valor de <b>n_neighbors</b>, ¿qué sucede ahora?
</div>

In [ ]:
knnModel = KNeighborsClassifier(n_neighbors=...).fit(X1, y1)
plot_decision_boundary(X1, y1, knnModel)

<div class = "alert alert-success">
EJERCICIO 1.4: Aplica el algoritmo K-NN sobre los ejemplos 2 y 3. ¿Qué sucedería si aplicamos sobre estos ejemplos un algoritmo de <a href="http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html">regresión logística</a>? ¿Qué pasa si variamos el número de vecinos?
</div>

In [ ]:
# Ejemplo 2
from sklearn.neighbors import KNeighborsClassifier

data2 = ejemplo2.values
X2 = ...
y2 = ...

# creamos el modelo y ajustamos
knnModel2 = KNeighborsClassifier(n_neighbors=...).fit(X2, y2)

plot_decision_boundary(X2, y2, knnModel2)

In [ ]:
from sklearn.linear_model import LogisticRegression

# .... código para crear el modelo y entrenar
lrModel = ...
plot_decision_boundary(X1, y1, lrModel)

In [ ]:
from sklearn.linear_model import LogisticRegression

# .... código para crear el modelo y entrenar
lrModel = ...
plot_decision_boundary(X2, y2, lrModel)

In [ ]:
# Ejemplo 3
from sklearn.neighbors import KNeighborsClassifier

# ... código para crear el modelo y entrenar
knnModel3 = ...

plot_decision_boundary(X3, y3, knnModel3)

In [ ]:
from sklearn.linear_model import LogisticRegression

# .... código para crear el modelo y entrenar
lrModel3 = ...
plot_decision_boundary(X3, y3, lrModel3)

Podemos comprobar que las mejores prestaciones se obtienen cuando *n_neighbors=1*, ¿tiene sentido? ¿Estamos midiendo correctamente las prestaciones de este clasificador?

# 3. Evaluación del modelo: entrenamiento y test

La respuesta es claramente no. Para poder saber cómo de bien se comporta un algoritmo de machine learning, hemos de medir su capacidad de [generalización](https://en.wikipedia.org/wiki/Generalization_error), esto es, las prestaciones en muestras no vistas previamente por el clasificador. Para ello, dividimos el conjunto de datos en dos partes, entrenamiento y test, teniendo en cuenta que:

![](./figuras/train_test_set_2d_classification.png)

* Utilizamos aproximadamente un 75-80% de las muestras para entrenamiento y un 25-20% para el test (cuidado! depende del tamaño del dataset; si es muy grande, el conjunto de test puede ser un porcentaje menor)
* Ambos conjuntos han de representar la población con la misma estadística: 
    * Randomizar, esto es, reordenar para evitar orden en las muestras. (cuidado series temporales)
    * Estratificar con respecto a una variable (normalmente la variable target), para mantener la proporción de la varible target en los conjuntos train/test.

sklern nos proporciona una [función](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) para dividir nuestros datos. 

Vamos a probar con el primer ejemplo:

In [ ]:
from sklearn.model_selection import train_test_split

print(type(X1))

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.3, shuffle=True)#, random_state=0)

print(type(X_train))

knn = KNeighborsClassifier(n_neighbors=15).fit(X_train, y_train)

plot_decision_boundary(X_test, y_test, knn)

<div class = "alert alert-success">
EJERCICIO 1.5: Sobre la celda anterior, varía el valor de <b>n_neighbors</b>. ¿Para qué valor se obtienen ahora las mejores prestaciones? ¿Qué sucede si eliminamos <b>random_state = 0</b> y ejecutamos varias veces la misma celda para un valor de <b>n_neighbors</b> fijo? ¿Obtenemos las mismas prestaciones?
</div>

<div class = "alert alert-success">
EJERCICIO 1.6: Calcula las prestaciones del algoritmo K-NN para los ejemplos 2 y 3. 
</div>

In [ ]:
# Ejemplo 2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(..., ..., test_size=0.3, shuffle=True, random_state=0)

knn = KNeighborsClassifier(n_neighbors=...).fit(X_train, y_train)

plot_decision_boundary(X_test, y_test, knn)

In [ ]:
# Ejemplo 3
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(..., ..., test_size=0.3, shuffle=True, random_state=0)

knn = KNeighborsClassifier(n_neighbors=...).fit(X_train, y_train)

plot_decision_boundary(X_test, y_test, knn)

<div class = "alert alert-success">
EJERCICIO 1.7: Vamos a realizar un análisis del parámetro de estratificación, para ver el efecto que tiene en los datos 
</div>

In [ ]:
# Una forma de estudiarlo
ejemplo1['label'].value_counts()

In [ ]:
# Otra forma de estudiarlo
np.unique(y_train, return_counts=True)

In [ ]:
y_train.shape

In [ ]:
# Ahora: cómo haríais el análisis completo?

# Porcentaje global
print('DISTRIBUCIÓN DEL DATASET ENTERO')
y = ejemplo1['label'].values
print(f'% ceros en total: {((np.unique(y, return_counts=True)[1][0]/y.shape[0])*100):.2f}')
print('\n')

# Sin estratificar
print('SIN ESTRATIFICAR')
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.3, shuffle = True)
print(f'% ceros en train: {((np.unique(y_train, return_counts=True)[1][0]/y_train.shape[0])*100):.2f}')
print(f'% ceros en test: {((np.unique(y_test, return_counts=True)[1][0]/y_test.shape[0])*100):.2f}')
print('\n')
      
# Estratificando
print('ESTRATIFICANDO')
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.3, shuffle = True, stratify=y1)
print(f'% ceros en train: {((np.unique(y_train, return_counts=True)[1][0]/y_train.shape[0])*100):.2f}')
print(f'% ceros en test: {((np.unique(y_test, return_counts=True)[1][0]/y_test.shape[0])*100):.2f}')
print('\n')

<div class = "alert alert-success">
EJERCICIO 1.8 (AVANZADO): Representa la performance del algoritmo K-NN en entrenamiento y test para distintos valores de <b>n_neighbors</b> (entre 1 y 15), utilizando el ejemplo 3. 
</div>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size = 0.3, shuffle = True, random_state = 0)

# inicializamos
neighbors = range(1,50)
acc_train = []
acc_test  = []

for n in neighbors:
    
    # ... código aquí
    # pista: lo único que hay que hacer es instanciar el modelo,
    # definiendo correctamente el parámetro del número de vecinos,
    # y luego hacer `.fit()` sobre los datos de train
    
    ...
    

plt.plot(neighbors,acc_train,'b',label='train')
plt.plot(neighbors,acc_test,'r',label='test')
plt.legend()
plt.xlabel('# vecinos')
plt.ylabel('ACC')
plt.show()


El número de vecinos que escojamos afecta significativamente a las prestaciones del algoritmo. Este parámetro es un compromiso entre los errores que cometemos (*accuracy*) y la complejidad del modelo (frontera de separación). 

- Cuanto menor es el número de vecinos, **más compleja** es la frontera de separación, y por tanto mayor será el sobreajuste. Potencialmente empeorará la *accuracy*.
- Cuanto mayor es el número de vecinos, **menos compleja** es la frontera de separación y por tanto menor será el sobreajuste. Potencialmente mejorará la *accuracy*.

## 3.1 Conclusiones

1. Si las muestras de entrenamiento son escasas (ejemplo 1), el error en test puede ser muy variable , dependiendo de las muestras incluidas en el conjunto de entrenamiento y el conjunto de test.

2. Las prestaciones (en test), dependen del número de vecinos que determinan la complejidad de la frontera de separación.

Teniendo en cuenta 1 y 2, ¿cómo puedo escoger el valor óptimo de *n_neighbors*?


# 4. Selección del modelo: validación cruzada

La validación cruzada (o cross-validation) consiste en subdivir el conjunto de entrenamiento en $K$ partes iguales, de tal forma que se utilizan $K-1$ para entrenar (ajustar el modelo) y el bloque $k$ restante para evaluar las prestaciones en función de los parámetros libres. Este proceso se repite $K$ veces (hasta que se barren todos los bloques) y los resultados se promedian.

Por suerte, no es necesario programar estas subdivisiones, porque scikit-learn tiene un clase que realiza este trabajo por nosotros. Puedes consultarlo [aquí](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).

Vamos a buscar el valor óptimo del número de vecinos utilizando una estrategia 5-fold CV.

In [ ]:
from sklearn.model_selection import StratifiedKFold

# recordemos que este es nuestro conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size = 0.3, shuffle = True, random_state = 0, stratify=y3)

nFolds = 5 #scikit-learn los llama splits
kf  = StratifiedKFold(n_splits = nFolds, shuffle = True, random_state=0)

nVecinos = range(1,16) # [1-15]

# inicializamos una matriz de errores, para cada valor de n_neighbors y cada iteración del algoritmo de cross-validation
# - tantas filas como número de folds
# - tantas columnas como valores de vector del numero de vecinos
accMatriz = np.zeros((nFolds,len(nVecinos))) 

j = 0 # inicializamos contador de columnas
for n in nVecinos:
       
    knn = KNeighborsClassifier(n_neighbors = n)
    
    i = 0 # inicializamos contador de filas
    for idxTrain, idxVal in kf.split(X_train,y_train):
      
        Xt = X_train[idxTrain,:]
        yt = y_train[idxTrain]
        Xv = X_train[idxVal,:]
        yv = y_train[idxVal]
        
        knn.fit(Xt,yt)
        accMatriz[i,j] = knn.score(Xv, yv) 
        
        i+=1
    j+=1

accVector = np.mean(accMatriz,axis=0)
accStd = np.std(accMatriz,axis=0)

In [ ]:
# Calculamos el valor óptimo
idx = np.argmax(accVector)
nOpt = nVecinos[idx]

plt.plot(nVecinos,accVector,'-o')
plt.plot(nVecinos[idx],accVector[idx],'rs')
plt.title('El número óptimo de vecinos es: %d' % nOpt)
plt.xlabel('# vecinos')
plt.ylabel('5-Fold ACC')
plt.grid()
plt.show()

Representemos ahora la gráfica anterior con la variación (desviación estándar) de la *accuracy* en cada *fold*. 

In [ ]:
plt.plot(nVecinos,accVector,'-o')
plt.plot(nVecinos[idx],accVector[idx],'rs')
plt.errorbar(nVecinos, accVector, yerr=accStd, ecolor='g')
plt.title('El número óptimo de vecinos es: %d' % nOpt)
plt.xlabel('# vecinos')
plt.ylabel('5-Fold ACC')
plt.grid()
plt.show()

In [ ]:
# Damos las prestaciones reales del modelo (en test)
knn = KNeighborsClassifier(n_neighbors = 15)
knn.fit(X_train,y_train)

print("accuracy: {:.2f}".format(knn.score(X_test, y_test)))

El código anterior se puede reducir drásticamente si utilizamos [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Pido perdón por el susto.

El código de la siguiente celda es equivalente al de las cuatro celdas anteriores:

In [ ]:
from sklearn.model_selection import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size = 0.3, shuffle = True, random_state = 0)

param_grid = {'n_neighbors':  np.arange(1, 16, 1)}
grid = GridSearchCV(KNeighborsClassifier(), scoring= 'accuracy', param_grid=param_grid, cv = 5, verbose=1).fit(X_train, y_train)
print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
print("best parameters: {}".format(grid.best_params_))

scores = np.array(grid.cv_results_['mean_test_score']) #¡cuidado, lo llaman test cuando es validación!
stdvalues = np.array(grid.cv_results_['std_test_score'])
plt.plot(np.arange(1, 16, 1),scores,'-o')
plt.errorbar(nVecinos, scores, yerr=stdvalues, ecolor='g')
plt.xlabel('# vecinos')
plt.ylabel('5-Fold ACC')
plt.grid()
plt.show()

print("acc (test): {:.2f}".format(grid.score(X_test, y_test)))

# Referencias

1. Capítulo 2. An Introduction to Statistical Learning. 
2. [Bias–variance tradeoff](https://en.wikipedia.org/wiki/Bias–variance_tradeoff)
3. [Underfitting and overfitting, scikit learn docs](http://scikit-learn.org/stable/auto_examples/model_selection/plot_underfitting_overfitting.html)

# Ejercicio final del tema

Aplica lo que has aprendido en un dataset de cyberseguridad. Vamos a utilizar para ello un dataset de deteccion de spam en correo electronico. En el dataset, cada fila corresponde con el analisis textual de un email en la bandeja de entrada de un empleado.

 - Analiza que tiene el dataset. Cuantas filas? Cuantas columnas? Cual es la variable objetivo? Que significa cada columna?
 - Que tipo de problema es? Clasificacion o regresion? Por que?
 - En base al tipo de problema detectado, elige un modelo de ML para entrenarlo sobre los datos.
 - Elige una metrica que tenga sentido para este problema.
 - Haz un split de los datos para el entrenamiento.
 - Entrena el modelo.
 - Evalua que tal ha ido el entrenamiento con la metrica que has utilizado.
 - Explica que significa el valor de la metrica obtenida a un niño de 5 años.
 - Usa el modelo entrenado para predecir sobre datos que no ha visto nunca (df_realWorld), tienen sentido las predicciones? Como puedes estar seguro?

In [163]:
df = pd.read_csv('data/spam.csv')
df_realWorld = pd.read_csv('data/spam_realWorld.csv')
df.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.0,3.537,40,191,1
1,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.0,3.537,40,191,1
2,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.00,0.223,0.0,0.000,0.000,0.0,3.000,15,54,1
3,0.00,0.00,0.00,0.0,1.88,0.00,0.00,1.88,0.00,0.00,...,0.00,0.206,0.0,0.000,0.000,0.0,2.450,11,49,1
4,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,0.00,...,0.04,0.030,0.0,0.244,0.081,0.0,1.729,43,749,1
